In [148]:
from importlib import reload
import logging

import pandas as pd

import import_excel, views

reload(logging)
reload(import_excel)
reload(views)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s:%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
pd.set_option('display.max_rows', 1000)

view = views.UFNSView()


/Users/maksimtunev/PycharmProjects/balanceTax/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
2023-04-07 15:00:22 INFO:Загрузка файла завершена. В платежах содержится 65 операций.
2023-04-07 15:00:22 INFO:Загрузка файла завершена. В платежах содержится 326 операций.
/Users/maksimtunev/PycharmProjects/balanceTax/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
2023-04-07 15:00:22 INFO:Загрузка файла завершена. В платежах содержится 637 операций.
2023-04-07 15:00:25 INFO:Загрузка файла завершена. В балансе содержится 816 операций.
2023-04-07 15:00:28 INFO:Загрузка файла завершена. В балансе содержится 1729 операций.
2023-04-07 15:00:30 INFO:Загрузка файла завершена

In [154]:
view.current_balance(payments=3543917.07, accrual=3200000)

-2852943.1999999997

In [155]:
view.model_balance()

,balance,credit,credit_correction,debit,debit_penni,debit_penni_291
Прибыль,-15642.00,0.00,0.00,15642.00,0.00,0.00
Страховые - Нетрудоспособность и Материнство,707856.63,1622335.98,950740.63,914479.35,44149.84,0.00
НДС,-182018.88,247477.95,0.00,668885.96,258662.96,239389.13
Страховые - ОМС (до до 1 января 2017),73.83,73.83,0.00,0.00,0.00,0.00
Страховые - Нетрудоспособность и Материнство (до 1 января 2017),0.00,0.00,0.00,0.00,0.00,0.00
Страховые - ОМС,-869350.08,998498.23,36251.52,2064908.35,305414.31,197060.04
Страховые - ПФР,1530429.06,8997857.89,4119553.08,7816725.36,740485.78,349296.53
НДФЛ,-4350392.96,3182648.80,0.00,7720061.53,978972.35,187019.77
Страховые - Доп. Тариф,-15055.87,23415.11,0.00,42929.28,6523.05,4458.30
Прибыль в ФЕД. Бюджет,-2760.00,0.00,0.00,2760.00,0.00,0.00


In [157]:
view.current_balance(payments=3543917.07, accrual=3200000)

-2852943.1999999997

In [147]:
# Проверить пени по НДФЛ попробовать найти оспоримое

df = view.df
mask = (df.operation_details == 'начислены пени (по расчету)') & (df.operation_date > pd.to_datetime('2021-06-30').date())
mask = mask & (df.deadline == pd.to_datetime('2021-06-30').date())
df[mask][view.COLUMNS_READABLE].sort_values(by='deadline').debit.sum()

977226.4900000001

In [146]:
# Что вообще произошло 09-22-2022

df = view.df
mask = (df.operation_date == pd.to_datetime('2022-09-22').date())
df[mask][view.COLUMNS_READABLE]

# Восстановили 9663903.18 вместе с новыми пенями

,operation_date,tax,operation_details,credit,debit,decision_number,document_period,document_number,deadline
1495,2022-09-22,Страховые - Нетрудоспособность и Материнство,списана задолженность,0.00,426158.29,NaN,nan,291,2021-06-30
1491,2022-09-22,Страховые - ПФР (не установлено),списана задолженность,0.00,38180.72,NaN,nan,291,2021-06-30
1492,2022-09-22,Страховые - Доп. Тариф,списана задолженность,0.00,20953.65,NaN,nan,291,2021-06-30
1493,2022-09-22,Страховые - ПФР,списана задолженность,0.00,633661.44,NaN,nan,291,2021-06-30
1494,2022-09-22,Страховые - Доп. Тариф,начислены пени (по расчету),0.00,4458.30,NaN,nan,,2021-06-30
1496,2022-09-22,Страховые - ПФР,начислены пени (по расчету),0.00,349296.53,NaN,nan,,2021-06-30
1503,2022-09-22,Страховые - Нетрудоспособность и Материнство,списана задолженность,0.00,67307.15,NaN,nan,291,2021-06-30
1498,2022-09-22,НДС,списана задолженность,0.00,1065299.59,NaN,nan,291,2021-06-30
1499,2022-09-22,Страховые - ОМС (до до 1 января 2017),списана задолженность,0.00,8482.71,NaN,nan,291,2021-06-30
1500,2022-09-22,Страховые - Нетрудоспособность и Материнство,списана задолженность,0.00,9034.38,NaN,nan,291,2021-06-30


In [142]:
df = view.correction_sum()
df.values.sum()

5106545.23

In [145]:
# Что вообще произошло 30-06-2022

df = view.df
mask = (df.operation_date == pd.to_datetime('2021-06-30').date())
df[mask][view.COLUMNS_READABLE].credit.sum()


8686679.41